# Feature Engineering - GUIDE Dataset

**Obiettivo:** Preparare il dataset GUIDE per il machine learning, creando features significative a livello Incident.

**Task:** Classificazione multi-classe di IncidentGrade (TruePositive, BenignPositive, FalsePositive)

**Approccio:**
1. Aggregare dati da Evidence → Alert → Incident level
2. Creare features numeriche e categoriche
3. Gestire valori mancanti
4. Codificare variabili categoriche
5. Salvare dataset processato per modeling

## 1. Setup e Caricamento Dati

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Librerie importate con successo!")

In [ ]:
# Carica il dataset pulito dall'EDA
file_path = '../data/GUIDE_Train.csv'

print("Caricamento dataset...")
df = pd.read_csv(file_path)

print(f"Dataset caricato: {df.shape[0]:,} righe, {df.shape[1]} colonne")
print(f"Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 2. Pulizia Iniziale

In [ ]:
# Rimuovi record senza target
print(f"Record con IncidentGrade nullo: {df['IncidentGrade'].isna().sum()}")
df = df[df['IncidentGrade'].notna()].copy()

# Rimuovi colonne con >97% missing
missing_pct = (df.isnull().sum() / len(df)) * 100
cols_to_drop = missing_pct[missing_pct > 97].index.tolist()
print(f"\nColonne rimosse (>97% missing): {cols_to_drop}")
df = df.drop(columns=cols_to_drop)

# Rimuovi duplicati su Id
duplicati = df['Id'].duplicated().sum()
print(f"\nDuplicati su Id: {duplicati}")
if duplicati > 0:
    df = df.drop_duplicates(subset=['Id'], keep='first')

print(f"\nDimensioni dopo pulizia: {df.shape}")

## 3. Parsing Temporale

In [ ]:
# Converti Timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hour'] = df['Timestamp'].dt.hour
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
df['TimeOfDay'] = pd.cut(df['Hour'], bins=[0, 6, 12, 18, 24], 
                         labels=['Night', 'Morning', 'Afternoon', 'Evening'], 
                         include_lowest=True)

print("Features temporali create")
print(f"Range temporale: {df['Timestamp'].min()} - {df['Timestamp'].max()}")

## 4. Aggregazione a Livello Incident

Questo è il passo più importante: trasformiamo le evidenze in features a livello incident.

In [ ]:
print("Creazione features aggregate per incident...\n")

# Funzione per ottenere il valore più comune (moda)
def get_mode(x):
    mode = x.mode()
    return mode[0] if len(mode) > 0 else x.iloc[0] if len(x) > 0 else None

# Aggregazioni per IncidentId
incident_agg = df.groupby('IncidentId').agg({
    # Target (sempre uguale per lo stesso incident)
    'IncidentGrade': 'first',
    
    # Conteggi strutturali
    'AlertId': 'nunique',           # Numero di alert nell'incident
    'Id': 'count',                   # Numero totale di evidenze
    'EntityType': 'nunique',         # Diversità di entity types
    'EvidenceRole': 'nunique',       # Diversità di evidence roles
    
    # Features categoriche (moda)
    'Category': get_mode,
    'DetectorId': get_mode,
    'OrgId': 'first',
    
    # Features temporali
    'Hour': ['min', 'max', 'mean'],
    'DayOfWeek': get_mode,
    'IsWeekend': 'max',
    'Timestamp': ['min', 'max'],     # Per calcolare durata
    
    # Features geografiche
    'CountryCode': 'nunique',
    'State': 'nunique',
    'City': 'nunique',
    
    # Features tecniche
    'DeviceId': 'nunique',
    'OSFamily': 'nunique',
    'OSVersion': 'nunique',
    
    # Features di sicurezza
    'SuspicionLevel': lambda x: x.notna().sum(),  # Quante evidenze hanno suspicion
    'LastVerdict': lambda x: x.notna().sum(),     # Quante hanno verdict
}).reset_index()

# Rinomina colonne multi-livello
incident_agg.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col 
                        for col in incident_agg.columns.values]

print(f"Dataset aggregato creato: {incident_agg.shape}")
print(f"\nPrime colonne: {list(incident_agg.columns[:10])}")

In [ ]:
# Calcola durata incident (differenza tra prima e ultima evidenza)
incident_agg['Duration_seconds'] = (
    pd.to_datetime(incident_agg['Timestamp_max']) - 
    pd.to_datetime(incident_agg['Timestamp_min'])
).dt.total_seconds()

# Rinomina colonne per chiarezza
rename_map = {
    'AlertId_nunique': 'NumAlerts',
    'Id_count': 'NumEvidences',
    'EntityType_nunique': 'NumEntityTypes',
    'EvidenceRole_nunique': 'NumEvidenceRoles',
    'Hour_min': 'Hour_First',
    'Hour_max': 'Hour_Last',
    'Hour_mean': 'Hour_Avg',
    'CountryCode_nunique': 'NumCountries',
    'State_nunique': 'NumStates',
    'City_nunique': 'NumCities',
    'DeviceId_nunique': 'NumDevices',
    'OSFamily_nunique': 'NumOSFamilies',
    'OSVersion_nunique': 'NumOSVersions',
    'SuspicionLevel_<lambda>': 'NumWithSuspicion',
    'LastVerdict_<lambda>': 'NumWithVerdict',
}

incident_agg = incident_agg.rename(columns=rename_map)

# Rimuovi colonne timestamp originali
incident_agg = incident_agg.drop(columns=['Timestamp_min', 'Timestamp_max'], errors='ignore')

print(f"Features ingegnerizzate: {incident_agg.shape[1] - 2}")  # -2 per IncidentId e target
incident_agg.head()

## 5. Feature Engineering Avanzato - MITRE Techniques

In [ ]:
# Analizza MITRE Techniques
print("Processing MITRE Techniques...")

# Crea features da MitreTechniques
mitre_features = df.groupby('IncidentId')['MitreTechniques'].agg([
    ('NumWithMitre', lambda x: x.notna().sum()),  # Quante evidenze hanno tecniche MITRE
    ('NumUniqueMitre', lambda x: len(set(','.join(x.dropna().astype(str)).split(',')))),  # Tecniche uniche
]).reset_index()

# Merge con dataset principale
incident_agg = incident_agg.merge(mitre_features, on='IncidentId', how='left')

print(f"Features MITRE aggiunte. Shape: {incident_agg.shape}")

## 6. Gestione Valori Categorici

In [ ]:
# Identifica colonne categoriche
categorical_cols = incident_agg.select_dtypes(include=['object', 'category']).columns.tolist()

# Rimuovi IncidentId e IncidentGrade
categorical_cols = [col for col in categorical_cols if col not in ['IncidentId', 'IncidentGrade']]

print(f"Colonne categoriche da processare: {categorical_cols}")

# Per colonne con alta cardinalità, sostituisci valori rari con 'Other'
for col in categorical_cols:
    if incident_agg[col].nunique() > 100:
        # Mantieni solo i top 50 valori
        top_values = incident_agg[col].value_counts().head(50).index
        incident_agg[col] = incident_agg[col].apply(lambda x: x if x in top_values else 'Other')
        print(f"  {col}: ridotto a {incident_agg[col].nunique()} categorie")

## 7. Preparazione per ML: Train/Test Split

In [ ]:
# Separa features e target
X = incident_agg.drop(columns=['IncidentId', 'IncidentGrade'])
y = incident_agg['IncidentGrade']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nDistribuzione target:")
print(y.value_counts())
print(f"\nProporzioni:")
print(y.value_counts(normalize=True))

In [ ]:
# Label encoding per variabili categoriche
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
X_encoded = X.copy()

for col in categorical_cols:
    if col in X_encoded.columns:
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
        label_encoders[col] = le

print(f"Encoded {len(label_encoders)} categorical features")
print(f"\nTipi di dati finali:")
print(X_encoded.dtypes.value_counts())

In [ ]:
# Gestisci missing values
print("\nMissing values prima dell'imputazione:")
missing = X_encoded.isnull().sum()
print(missing[missing > 0])

# Sostituisci NaN con -999 (XGBoost gestisce bene questo approccio)
X_encoded = X_encoded.fillna(-999)

print(f"\nMissing values dopo imputazione: {X_encoded.isnull().sum().sum()}")

In [ ]:
# Split stratificato
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)

print("Train/Test Split completato")
print(f"\nX_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"\nDistribuzione y_train:")
print(y_train.value_counts(normalize=True))
print(f"\nDistribuzione y_test:")
print(y_test.value_counts(normalize=True))

## 8. Salvataggio Dataset Processato

In [ ]:
# Salva i dataset processati
import pickle

# Crea directory per i dati processati
import os
os.makedirs('../data/processed', exist_ok=True)

# Salva train/test
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False, header=['IncidentGrade'])
y_test.to_csv('../data/processed/y_test.csv', index=False, header=['IncidentGrade'])

# Salva label encoders
with open('../data/processed/label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

# Salva anche il dataset aggregato completo
incident_agg.to_csv('../data/processed/incident_features.csv', index=False)

print("Dataset salvati in ../data/processed/")
print(f"  - X_train.csv: {X_train.shape}")
print(f"  - X_test.csv: {X_test.shape}")
print(f"  - y_train.csv: {y_train.shape}")
print(f"  - y_test.csv: {y_test.shape}")
print(f"  - incident_features.csv: {incident_agg.shape}")

## 9. Analisi Features Create

In [ ]:
# Lista features create
print(f"Totale features: {X_encoded.shape[1]}")
print(f"\nLista completa features:")
for i, col in enumerate(X_encoded.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Statistiche descrittive delle features numeriche principali
key_features = ['NumAlerts', 'NumEvidences', 'NumEntityTypes', 'NumDevices', 
                'Duration_seconds', 'NumWithMitre', 'NumCountries']

available_features = [f for f in key_features if f in X_encoded.columns]
X_encoded[available_features].describe()

In [ ]:
# Visualizza distribuzione di alcune features chiave per target
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

plot_features = ['NumAlerts', 'NumEvidences', 'NumEntityTypes', 
                 'NumDevices', 'Duration_seconds', 'NumWithMitre']

for i, feature in enumerate(plot_features):
    if feature in incident_agg.columns:
        for grade in incident_agg['IncidentGrade'].unique():
            data = incident_agg[incident_agg['IncidentGrade'] == grade][feature]
            axes[i].hist(data, alpha=0.5, label=grade, bins=30)
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Frequenza')
        axes[i].legend()
        axes[i].set_title(f'Distribuzione {feature} per IncidentGrade')

plt.tight_layout()
plt.show()

## 10. Riepilogo Feature Engineering

**Features create:**
1. **Conteggi strutturali:** NumAlerts, NumEvidences, NumEntityTypes, NumEvidenceRoles
2. **Features temporali:** Hour_First, Hour_Last, Hour_Avg, DayOfWeek, IsWeekend, Duration_seconds
3. **Features geografiche:** NumCountries, NumStates, NumCities
4. **Features tecniche:** NumDevices, NumOSFamilies, NumOSVersions
5. **Features di sicurezza:** NumWithSuspicion, NumWithVerdict, NumWithMitre, NumUniqueMitre
6. **Features categoriche:** Category, DetectorId, OrgId (encoded)

**Prossimi passi:**
- Training modelli (XGBoost, LightGBM, Random Forest)
- Ottimizzazione iperparametri
- Feature importance analysis
- Cross-validation con macro-F1 score